In [1]:
from os import listdir
import pandas as pd
from bs4 import BeautifulSoup
from requests import get
import numpy as np
from tqdm import tqdm
import wikipedia
from nltk.metrics.distance import edit_distance

In [2]:
MISSING = ""

# Loading local data

In [192]:
#Récupére les données sur le PC pour éviter de scrapper à chaque fois
files = listdir("../chart")

dfs_chart = {}
for f in files:
    year = f.split("_")[0]
    month = f.split("_")[1].split(".")[0]

    if dfs_chart.get(year) is None:
        dfs_chart[year] = {}
        
    dfs_chart[year][month] = pd.read_csv("../chart/" + f)

In [193]:
dfs_chart["2006"]["Janvier"].head(3)

,Rank,Artist,Music,Artist_wiki,Naissance,Pays d'origine,Origine,Nationalité,Pays,Summary
0,1,Juanes,La Camisa Negra,Juanes,"9 août 1972 (50 ans)Medellin, Colombie",NaN,NaN,NaN,NaN,"Juan Esteban Aristizábal Vásquez, alias Juanes..."
1,2,Madonna,Hung Up,Madonna,"16 août 1958 (64 ans)Bay City, Michigan (États...",NaN,NaN,Américaine,NaN,"Madonna Louise Ciccone, dite Madonna, née le 1..."
2,3,Johnny Hallyday,Mon Plus Beau Noël,Johnny Hallyday,15 juin 1943 Paris 9e (France),NaN,NaN,Française,NaN,"Jean-Philippe Smet, dit Johnny Hallyday, né le..."


In [135]:
artist = pd.read_csv("../artist.csv")

In [136]:
artist

,Artist,Artist_wiki,Naissance,Pays d'origine,Origine,Nationalité,Pays,Summary
0,LP,LP,NaN,NaN,NaN,NaN,NaN,NaN
1,Les Enfants Du Pays,Les Enfants du pays,NaN,NaN,NaN,française,france,Les Enfants du pays est un film français de Pi...
2,Lynda,Lynda,NaN,NaN,NaN,NaN,NaN,"Lynda Carter, née le 24 juillet 1951 à Phoenix..."
3,Iv My People,IV My People,NaN,france,NaN,française,france,IV My People est un label discographique et co...
4,Mickael Turtle,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2267,Andy Williams,Andy Williams (chanteur),"3 décembre 1927Wall Lake, (Iowa, États-Unis)",NaN,NaN,NaN,NaN,"Howard Andrew Williams, né le 3 décembre 1927,..."
2268,Duffy,Duffy (chanteuse),"23 juin 1984 (38 ans) Bangor, pays de Galles",NaN,NaN,NaN,NaN,"Aimée Ann Duffy, alias Duffy, est une chanteus..."
2269,Martial,Martial,NaN,NaN,NaN,NaN,NaN,NaN
2270,Lacrim,Lacrim,"19 avril 1985 (37 ans)Paris (20e), France",NaN,NaN,française,NaN,"Lacrim, de son vrai nom Karim Zenoud, est un r..."


# Supression des dernières columns

In [194]:
#Supprime les dernières colonnes pour relancer le scrapping sur wikipédia
for k1 in dfs_chart.keys():
    for k2 in dfs_chart[k1].keys():
        dfs_chart[k1][k2].drop(columns=["Naissance", "Pays d'origine", "Origine", "Pays", "Nationalité", "Artist_wiki", "Summary"], inplace=True)

In [195]:
dfs_chart["2006"]["Janvier"].head(3)

,Rank,Artist,Music
0,1,Juanes,La Camisa Negra
1,2,Madonna,Hung Up
2,3,Johnny Hallyday,Mon Plus Beau Noël


In [139]:
artist.drop(columns=["Naissance", "Pays d'origine", "Origine", "Pays", "Nationalité"], inplace=True)
artist.head(3)

,Artist,Artist_wiki,Summary
0,LP,LP,NaN
1,Les Enfants Du Pays,Les Enfants du pays,Les Enfants du pays est un film français de Pi...
2,Lynda,Lynda,"Lynda Carter, née le 24 juillet 1951 à Phoenix..."


In [140]:
artist.drop(columns=["Artist_wiki"], inplace=True)

# Scrapping chart

In [3]:
years = [str(y) for y in np.arange(2004, 2023, 1)]
weeks = [int(v) for v in np.linspace(1, 52, 12)]
mois = ["Janvier", "Fevrier", "Mars", "Avril", "Mai", "Juin", "Juillet", "Aout", "Septembre", "Octobre", "Novembre", "Decembre"]

In [41]:
dfs_chart = {y : {} for y in years}
for y in tqdm(years):
    for k, w in enumerate(weeks):
        url = f"https://acharts.co/france_singles_top_100/{y}/{w}"
        rq = get(url)
        m = mois[k]
        if rq.ok:
            soup = BeautifulSoup(rq.text)

            chart = soup.find("table", {"id" : "ChartTable"})
            trs = chart.findAll("tr")[1:]

            data = []
            for tr in trs:
                music_name = tr.find("span", {"itemprop" : "name"}).text.lower()
                rank = tr.find("span", {"itemprop" : "position"}).text.lower()
                artist_name = tr.find("span", {"itemprop" : "byArtist"}).text[2:-1].lower()
                data.append([str(rank), str(artist_name), str(music_name)])

            df = pd.DataFrame(data, columns=["Rank", "Artist", "Music"])
            dfs_chart[y][m] = df

100%|██████████| 19/19 [01:13<00:00,  3.85s/it]


In [42]:
names = [
    ("Djadja", "Djadja et Dinaz"),
    ("Lorie", "Lorie Pester"),
    ("-M-", "Matthieu Chedid"),
    ("Priscilla", "Priscilla Betti"),
    ("I Am", "IAM"),
    ("Sofiane", "Sofiane Zermani"),
    ("Justice", "Justice (groupe"),
    ("Soma Riba", "Collectif Métissé"),
    ("Fresh", "Fresh la Peufra"),
    ("Rosalia", "Rosalía"),
    ("Italo Brothers", "ItaloBrothers"),
    ("Far*east Movement", "Far East Movement"),
    ("Odyssey", "Odyssey (groupe)"),
    ("1789", "1789 : Les Amants de la Bastille"),
    ("Clemence", "Clémence Saint-Preux"),
    ("\x9c", "oe"),
    ("œ", "oe"),
    ("Rose", "Rose (chanteuse)"),
    ("Laeti", "Laetitia Kerfa"),
    ("La Troupe", "Mozart, l'opéra rock"),
    ("Victoria", "Victoria Sio"),
    ("Christine And The Queens", "Redcar (artiste)"),
    ("Earth and Wind And Fire", "Earth, Wind And Fire"),
    ("Karol G", "Karol"),
    ("$-Crew", "S-Crew"),
    ("Jean Roch", "Jean-Roch"),
    ("Keen V", "Keen'V"),
    ("DJ Khaled", "Khaled"),
    ("The Black Eyed Peas", "Black Eyed Peas"),
    ("Dinor", "Dinor RDT"),
    ("Koba La D", "Koba LaD")
]

for k1 in dfs_chart.keys():
    for k2 in dfs_chart[k1].keys():
        dfs_chart[k1][k2]["Artist"] = dfs_chart[k1][k2]["Artist"].str.split(" x ").str[0]
        dfs_chart[k1][k2]["Artist"] = dfs_chart[k1][k2]["Artist"].str.split(" - ").str[-1]
        dfs_chart[k1][k2]["Artist"] = dfs_chart[k1][k2]["Artist"].str.split(" \+ ").str[0]
        
        for n1, n2 in names:
            dfs_chart[k1][k2]["Artist"].replace(n1.lower(), n2.lower(), inplace=True)
        

# Scrapping birth

In [43]:
def find_title_in_wikipedia(title):
    pourcentage = 0.3
    words = ["(chanteur)", "(chanteuse)", "(groupe)", "(rappeur)", "(rappeuse)", "(musicien)", "(chanteur français)", "(france)", "(producteur)", "(artiste)", "(groupe de musique)"]

    wikipedia.set_lang("fr")
    results = wikipedia.search(title, results=10)
    distance = []
    if len(results) > 0:
        for element in results:
            if any((w in element.lower()) and (edit_distance(element.lower().split("(")[0][:-1], title.lower())/len(title) < pourcentage) for w in words):
                return element

            distance.append(edit_distance(title.lower(), element.lower()))

        return results[np.argmin(distance)] if min(distance)/len(title) < pourcentage else MISSING

In [44]:
def wiki_birth(title):
    cols = ["Naissance", "Pays d'origine", "Origine", "Nationalité", "Pays", "Summary"]
    nats = ["franco", "français", "belge", "canadien", "libanais", "réunionnais"]
    dic = {w : MISSING for w in cols}

    if title == MISSING:
        return dic

    url = f"https://fr.wikipedia.org/wiki/{title}"
    rq = get(url)

    if not rq.ok:
        return dic
    
    soup = BeautifulSoup(rq.text)
    tables = soup.findAll("table")

    for table in tables:
        trs = table.findAll("tr")

        for tr in trs:
            th = tr.find("th")

            if th is not None:
                for w in cols:
                    if w in th.text:
                        td = tr.find("td")
                        if td is not None:
                            dic[w] = td.text.strip().lower()

    wikipedia.set_lang("fr")
    try:
        summary = wikipedia.summary(title, sentences=1)
        dic["Summary"] = summary.lower()

        if dic["Nationalité"] == MISSING:
            for w in nats:
                if w in summary:
                    dic["Nationalité"] = w.lower()
                    return dic
    except:
        pass

    return dic

In [45]:
#Récupére tous les artistes uniques
artist = []
for k1 in dfs_chart.keys():
    for k2 in dfs_chart[k1].keys():
        artist += dfs_chart[k1][k2]["Artist"].tolist()
    
artist = pd.DataFrame(list(set(artist)), columns=["Artist"])

In [46]:
artist["Artist_wiki"] = artist["Artist"].apply(find_title_in_wikipedia) #Trouve les pages wikipedia de chaque artistes

In [47]:
birth_dic = artist["Artist_wiki"].apply(wiki_birth) #Cherche les infos de naissance sur les pages wikipedia

c:\Users\quent\AppData\Local\Programs\Python\Python310\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\quent\AppData\Local\Programs\Python\Python310\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [48]:
#Transforme les infos trouver sur wikipédia en dataframe
dfs_birth = []
for dic in birth_dic:
    dfs_birth.append(pd.DataFrame(dic, index=[0]))
birth = pd.concat(dfs_birth, ignore_index=True)

In [49]:
#Merge les infos de naissance avec les infos des artistes
artist = artist.merge(birth, left_index=True, right_index=True)
artist = artist.set_index("Artist")

for k1 in dfs_chart.keys():
    for k2 in dfs_chart[k1].keys():
        dfs_chart[k1][k2] = dfs_chart[k1][k2].merge(artist, on="Artist", how="left")

# CLEANNING

In [50]:
def cleanning(data, replace_words):
    new_df = data.copy()
    new_df = new_df.str.lower()
    for w1, w2 in replace_words:
        new_df[new_df.str.contains(w1)] = w2

    return new_df

In [51]:
replace_words = [
    ("français", "française"),
    ("franco", "française"),
    ("canadien", "canadienne"),
    ("américain", "américaine"),
    ("états-unis", "américaine"),
    ("algérien", "algérienne"),
    ("brésil", "brésilienne"),
    ("france", "française"),
    ("marocain", "marocaine"),
    ("anglais", "britannique"),
    ("royaume-unis", "britannique"),
    ("britannique", "britannique"),
    ("espagn", "espagnole"),
    ("italie", "italienne"),
    ("japon", "japonaise"),
    ("corée", "coréenne"),
    ("israél", "israélienne"),
    ("royaume-uni", "britannique"),
    ("suède", "suédoise"),
    ("allemand", "allemande"),
    ("dominicain", "dominicaine")
]

artist["Nationalité"] = cleanning(artist["Nationalité"], replace_words)
artist["Origine"] = cleanning(artist["Origine"], replace_words)

In [52]:
replace_words = [
    ("france", "france"),
    ("états-unis", "états-unis"),
    ("royaume-uni", "royaume-unis"),
    ("allemagne", "allemagne"),
    ("angleterre", "angleterre"),
    ("canada", "canada"),
    ("australie", "australie"),
    ("pays-bas", "pays-bas"),
    ("ghana", "ghana"),
    ("corée", "corée"),
    ("autriche", "autriche"),
    ("suède", "suède"),
    ("italie", "italie"),
    ("biélorussie", "biélorussie"),
    ("mauritanie", "mauritanie")
]

artist["Pays"] = cleanning(artist["Pays"], replace_words)
artist["Pays d'origine"] = cleanning(artist["Pays d'origine"], replace_words)

# Birthday case

# Export

In [53]:
for k1 in dfs_chart.keys():
    for k2 in dfs_chart[k1].keys():
        dfs_chart[k1][k2].to_csv(f"../chart/{k1}_{k2}.csv", encoding="utf-8-sig", index = False)

In [54]:
artist.to_csv("../artist.csv", encoding="utf-8-sig")